## nn.Module vs. nn.Sequential
nn.Module은 딥러닝에 사용되는 모든 파라미터가 저장되어 있고 자신이 원하는 대로 신경망 동작 정의 가능
nn.Sequential은 층을 쌓기만 하는 간단한 구조에 사용하기 편리 그러나 은닉층에서 순전파 도중의
결과를 저장한다거나 데이터 흐름을 제어하는 등의 커스터마이징 불가능

In [1]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_dim):
        #자식 클래스가 상속받은 부모 클래스를 자식클래스에 불러오겠다 
        #super().__init__()과 차이 없음 현재 사용하는 클래스가 무엇인지 알려주는 용도임 
        super(BasicBlock, self).__init__()
        
        #합성곱층 정의
        self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels,kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        return x

합성곱을 하여 나온 이미지를 평탄화(1차원 벡터로 변환)
MLP층은 벡터만을 입력으로 받을 수 있기 때문에 이 작업이 필요


In [2]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        # 합성곱 기본 블록 정의
        self.block1 = BasicBlock(in_channels=3, out_channels=32, hidden_dim=16)
        self.block2 = BasicBlock(in_channels=32, out_channels=128, hidden_dim=64)
        self.block3 = BasicBlock(in_channels=128, out_channels=256, hidden_dim=128)
        
        # 분류기 정의
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, start_dim=1)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

In [7]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import RandomHorizontalFlip, ToTensor, Normalize
from torchvision.transforms import Compose
from torchvision.transforms import RandomCrop
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam

transforms = Compose([
    RandomCrop((32,32), padding=4), #랜덤 크롭핑
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=(0.247, 0.243, 0.261)),
])

training_data = CIFAR10(
    root="./",
    train=True,
    download=True,
    transform=transforms
)

test_data = CIFAR10(
    root="./",
    train=False,
    download=True,
    transform=transforms
)

train_loader = DataLoader(training_data, batch_size=32,shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNN(num_classes=10).to(device)

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)

for epoch in range(100):
    for data, label in train_loader:
        optim.zero_grad()
        
        preds = model(data.to(device))
        
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()
        
    if epoch == 0 or epoch%10 == 9:
        print(f"epoch{epoch+1} loss:{loss.item()}")
        
torch.save(model.state_dict(),"CIFAR.pth")
        

Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 

In [8]:
#모델 성능 평가하기
model.load_state_dict(torch.load("CIFAR.pth", map_location=device))

num_corr = 0

with torch.no_grad():
    for data, label in test_loader:
        
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr
        
    print(f"Accuracy: {num_corr/len(test_data)}")

FileNotFoundError: [Errno 2] No such file or directory: 'CIFAR.pth'

이미지 개수가 많을 수록 더 오랜 학습시간이 필요 
데이터가 달라질 때마다 매번 새로 학습해야 할까? 
## 전이학습
다른 데이터를 이용해 학습된 모델을 갖고 있는 데이터에 최적화 시키는 학습 방법
읿반적으로 대용량 데이터를 이요해 사전 학습한 모델을 소규모 데이터에 최적화 시킨다